In [1]:
import warnings
warnings.filterwarnings("ignore")

%load_ext autoreload
%autoreload 2


In [4]:
from qiskit_metal import Dict, MetalGUI, designs
from qiskit_metal.qlibrary.qubits.transmon_cross import TransmonCross
from qiskit_metal.qlibrary.qubits.transmon_pocket import TransmonPocket
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight
from qiskit_metal.qlibrary.terminations.launchpad_wb_coupled import LaunchpadWirebondCoupled
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee

In [3]:
design = designs.DesignFlipChip({}, overwrite_enabled=True)

# Uncomment to launch Qiskit Metal GUI
gui = MetalGUI(design)

In [6]:
# Design variables
center_x =0
center_y =0
x = 0.6875+0.650 # qubit x spacing
y = 2.7+0.650 # qubit y spacing


In [25]:
connection_pads_l= dict(c = dict(loc_W=1, loc_H=-1, pad_width='120um'),
                       a  = dict(loc_W=-1, loc_H=+1, pad_width='120um'),
                       b = dict(loc_W = -1, loc_H = -1, pad_width='120um', pocket_rise='100um'))
q5=[TransmonPocket(design,'Q5', options=dict(pos_x=f"{center_x-2*x}mm",
                                                    pos_y=f"{center_y-y/2}",
                                                    chip="Q_chip",
                                               connection_pads=connection_pads_l))]                                    
gui.rebuild()
gui.autoscale()

In [4]:
from qiskit_metal.qlibrary.terminations.open_to_ground import OpenToGround
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
open_start_options = Dict(pos_x='000um',
                         pos_y='0um',
                         orientation = '-90')

open_start_meander = OpenToGround(design,'Open_meander_start',options=open_start_options)





open_end_options = Dict(pos_x='000um',
                        pos_y='1500um',
                        orientation='90',
                        termination_gap='10um')

open_end_meander = OpenToGround(design,'Open_meander_end',options=open_end_options)

In [5]:
meander_options = Dict(pin_inputs=Dict(start_pin=Dict(
                                            component='Open_meander_start',
                                            pin='open'),
                                        end_pin=Dict(
                                            component='Open_meander_end',
                                            pin='open')
                                        ),
                      total_length='7mm',
                      meander=Dict(asymmetry='-5mm'),
                      fillet='99.99um')

testMeander = RouteMeander(design,'meanderTest',options=meander_options)
gui.rebuild()
gui.autoscale()
gui.zoom_on_components([testMeander.name, open_start_meander.name, open_end_meander.name])

05:09PM 40s WARNING [check_lengths]: For path table, component=meanderTest, key=trace has short segments that could cause issues with fillet. Values in (1-1)  are index(es) in shapely geometry.
05:09PM 40s WARNING [check_lengths]: For path table, component=meanderTest, key=cut has short segments that could cause issues with fillet. Values in (1-1)  are index(es) in shapely geometry.
05:09PM 40s WARNING [check_lengths]: For path table, component=meanderTest, key=trace has short segments that could cause issues with fillet. Values in (1-1)  are index(es) in shapely geometry.
05:09PM 40s WARNING [check_lengths]: For path table, component=meanderTest, key=cut has short segments that could cause issues with fillet. Values in (1-1)  are index(es) in shapely geometry.


In [6]:
# Design variables
center_x = center_y = 0
qubit_spacing_x = 2
qubit_spacing_y = 4
offset = qubit_spacing_x//2

feed_x = qubit_spacing_x*1.5
design.delete_all_components()

In [7]:
# Qubits
conn_pad = dict(connector_type="0",
               claw_length="100um",
               claw_width="15um")

q1 = [TransmonCross(design, "Q1", options=dict(pos_x=f"{center_x-qubit_spacing_x//2 - offset}mm",
                                               pos_y=f"{center_y-qubit_spacing_y//2}mm",
                                               chip="Q_chip",
                                               cross_width="40um"))]

q2 = [TransmonCross(design, "Q2", options=dict(pos_x=f"{center_x+qubit_spacing_x//2 - offset}mm",
                                               pos_y=f"{center_y-qubit_spacing_y//2}mm",
                                               chip="Q_chip",
                                               cross_width="40um"))]

q3 = [TransmonCross(design, "Q3", options=dict(pos_x=f"{center_x-qubit_spacing_x//2}mm",
                                               pos_y=f"{center_y+qubit_spacing_y//2}mm",
                                               chip="Q_chip",
                                               cross_width="40um",
                                               orientation="180"))]

q4 = [TransmonCross(design, "Q4", options=dict(pos_x=f"{center_x+qubit_spacing_x//2}mm",
                                               pos_y=f"{center_y+qubit_spacing_y//2}mm",
                                               chip="Q_chip",
                                               cross_width="40um",
                                               orientation="180"))]

qubits = [q1, q2, q3, q4]

# Uncomment when not using GUI, comment otherwise
design.rebuild()
gui.rebuild()
gui.autoscale()

In [8]:
# Launchpads
lwc_left = LaunchpadWirebondCoupled(design, "lwc_left",
                                    options=dict(pos_x=f"-{feed_x}mm",
                                                 pos_y="0mm",
                                                 chip="C_chip"))

lwc_right = LaunchpadWirebondCoupled(design, "lwc_right",
                                     options=dict(pos_x=f"{feed_x-offset}mm",
                                                  pos_y="0mm",
                                                  chip="C_chip",
                                                  orientation="180"))

# Uncomment when not using GUI, comment otherwise
design.rebuild()

# Uncomment when using GUI, comment otherwise
# gui.rebuild()
# gui.autoscale()

In [9]:
# Coupled line tee
clt1 = CoupledLineTee(design, "clt1", options=dict(pos_x=f"{center_x-qubit_spacing_x//2-offset}", chip="C_chip",))
clt2 = CoupledLineTee(design, "clt2", options=dict(pos_x=f"{center_x-qubit_spacing_x//2}", chip="C_chip", orientation="180"))
clt3 = CoupledLineTee(design, "clt3", options=dict(pos_x=f"{center_x+qubit_spacing_x//2-offset}", chip="C_chip",))
clt4 = CoupledLineTee(design, "clt4", options=dict(pos_x=f"{center_x+qubit_spacing_x//2}", chip="C_chip", orientation="180"))

# Uncomment when not using GUI, comment otherwise
design.rebuild()

# Uncomment when using GUI, comment otherwise
# gui.rebuild()
# gui.autoscale()

In [10]:
# Transmission lines
fl1 = RouteStraight(design, "fl1", options=dict(
                                        pin_inputs=dict(
                                            start_pin=dict(component="lwc_left", pin="tie"),
                                            end_pin=dict(component="clt1", pin="prime_start")
                                        ),
                                        chip="C_chip"))
                    
fl2 = RouteStraight(design, "fl2", options=dict(
                                        pin_inputs=dict(
                                            start_pin=dict(component="clt1", pin="prime_end"),
                                            end_pin=dict(component="clt2", pin="prime_end")
                                        ),
                                        chip="C_chip"))

fl3 = RouteStraight(design, "fl3", options=dict(
                                        pin_inputs=dict(
                                            start_pin=dict(component="clt2", pin="prime_start"),
                                            end_pin=dict(component="clt3", pin="prime_start")
                                        ),
                                        chip="C_chip"))

fl4 = RouteStraight(design, "fl4", options=dict(
                                        pin_inputs=dict(
                                            start_pin=dict(component="clt3", pin="prime_end"),
                                            end_pin=dict(component="clt4", pin="prime_end")
                                        ),
                                        chip="C_chip"))

fl5 = RouteStraight(design, "fl5", options=dict(
                                        pin_inputs=dict(
                                            start_pin=dict(component="clt4", pin="prime_start"),
                                            end_pin=dict(component="lwc_right", pin="tie")
                                        ),
                                        chip="C_chip"))

# Uncomment when not using GUI, comment otherwise
design.rebuild()

# Uncomment when using GUI, comment otherwise
gui.rebuild()
gui.autoscale()

In [11]:
# Readout couplers
from qiskit_metal.qlibrary.user_components.quad_coupler import QuadCoupler

qc1 = QuadCoupler(design, "qc1", options=dict(pos_x=f"{center_x-qubit_spacing_x/2-offset}mm",
                                              pos_y=f"{center_y-qubit_spacing_y/2+0.18}mm",
                                              pad_width="60um",
                                              pad_height="80um",
                                              orientation="180",
                                              chip="C_chip"))

qc2 = QuadCoupler(design, "qc2", options=dict(pos_x=f"{center_x+qubit_spacing_x/2-offset}mm",
                                              pos_y=f"{center_y-qubit_spacing_y/2+0.18}mm",
                                              pad_width="60um",
                                              pad_height="80um",
                                              orientation="180",
                                              chip="C_chip"))

qc3 = QuadCoupler(design, "qc3", options=dict(pos_x=f"{center_x-qubit_spacing_x/2}mm",
                                              pos_y=f"{center_y+qubit_spacing_y/2-0.18}mm",
                                              pad_width="60um",
                                              pad_height="80um",
                                              chip="C_chip"))

qc4 = QuadCoupler(design, "qc4", options=dict(pos_x=f"{center_x+qubit_spacing_x/2}mm",
                                              pos_y=f"{center_y+qubit_spacing_y/2-0.18}mm",
                                              pad_width="60um",
                                              pad_height="80um",
                                              chip="C_chip"))

# Uncomment when not using GUI, comment otherwise
design.rebuild()

# Uncomment when using GUI, comment otherwise
gui.rebuild()
gui.autoscale()

In [ ]:
# Meandered readouts
coup_opts = dict(
    chip="C_chip",
    fillet="99.99um",
    lead = dict(start_straight="200um")
)

ro1 = RouteMeander(design, "ro1", options=dict(
                                            pin_inputs=dict(
                                                start_pin=dict(component="qc1", pin="in"),
                                                end_pin=dict(component="clt1", pin="second_end"),
                                            ),
                                            **coup_opts))

ro2 = RouteMeander(design, "ro2", options=dict(
                                            pin_inputs=dict(
                                                end_pin=dict(component="qc2", pin="in"),
                                                start_pin=dict(component="clt3", pin="second_end"),
                                            ),
                                            **coup_opts))

ro3 = RouteMeander(design, "ro3", options=dict(
                                            pin_inputs=dict(
                                                start_pin=dict(component="qc3", pin="in"),
                                                end_pin=dict(component="clt2", pin="second_end"),
                                            ),
                                            **coup_opts))

ro4 = RouteMeander(design, "ro4", options=dict(
                                            pin_inputs=dict(
                                                start_pin=dict(component="qc4", pin="in"),
                                                end_pin=dict(component="clt4", pin="second_end"),
                                            ),
                                            **coup_opts))

# Uncomment when not using GUI, comment otherwise
design.rebuild()

# Uncomment when using GUI, comment otherwise
gui.rebuild()
gui.autoscale()

In [ ]:
from qiskit_metal.renderers.renderer_gmsh.gmsh_renderer import QGmshRenderer

# Initialize Gmsh renderer
gmsh_renderer = QGmshRenderer(design)

# Set initial constraints for the mesh
gmsh_renderer.options.mesh.min_size = "5um"
gmsh_renderer.options.mesh.max_size = "80um"

In [ ]:
# Render the design in Gmsh
# skip_junctions: always set to True for now
# mesh_geoms: set to True if you want to mesh the geometries, otherwise False
gmsh_renderer.render_design(skip_junctions=True, mesh_geoms=False)

In [ ]:
# Launch the Gmsh GUI
gmsh_renderer.launch_gui()

# close the Gmsh renderer
gmsh_renderer.close()